In [1]:
import requests
from requests import get
from bs4 import BeautifulSoup

import pandas as pd
import numpy as np

import re

from time import sleep
from random import randint

In [2]:
# Movies in English
headers = {"Accept-Language": "en-US, en;q=0.5"}

# Request contents of the URL
# https://www.imdb.com/search/title/?release_date=1926-01-01,2021-12-31&ref_=adv_prv
url = "https://www.imdb.com/search/title/?groups=top_1000&sort=moviemeter,desc"
results = requests.get(url, headers=headers) 

# Using BeautifulSoup
soup = BeautifulSoup(results.text, "html.parser")

# Find all lister-item mode-advanced divs
movie_div = soup.find_all('div', class_='lister-item mode-advanced')

In [3]:
6701765
pages = np.arange(1, 1001, 50)
pages

array([  1,  51, 101, 151, 201, 251, 301, 351, 401, 451, 501, 551, 601,
       651, 701, 751, 801, 851, 901, 951])

In [4]:
#initialize empty lists where you'll store your data

ids = []
budgets = []
genres = []
original_languages = []
overviews = []
    ##
popularity = []
    ##
production_companies = []
release_dates = []
    ##
revenue = []
runtimes = []
    ##
tagline = []
titles = []
imdb_ratings = []
vote_counts = []

    # Creating the lists we want to write into

metascores = []
movieDirector = []
movieStars = []
posters = []
urls = []

meanImdb_rating = 6.9
minVote_count = 3000

In [5]:
# Storing each of the urls of  movies 
for page in pages:
    # Getting the contents from the each url
    page = requests.get('https://www.imdb.com/search/title/?groups=top_1000&start=' + str(page) + '&ref_=adv_nxt', headers=headers)
    soup = BeautifulSoup(page.text, 'html.parser')
    # Aiming the part of the html we want to get the information from
    movie_div = soup.find_all('div', class_='lister-item mode-advanced')
    # Controling the loop’s rate by pausing the execution of the loop for a specified amount of time
    # Waiting time between requests for a number between 2-10 seconds
    #sleep(randint(2,10))
    for container in movie_div:
        # Scraping the movie's id
        s_id = container.h3.a['href'].split('/')[2]
        ids.append(s_id)
        
        # Scraping the movie's url
        url ='http://www.imdb.com/title/' + container.h3.a['href'].split('/')[2]
        urls.append(url)
        
        # Scraping the movie's id
        # Scraping the movie's id
        try:
            image_url = container.find('div', class_='lister-item-image float-left').img['loadlate']
        except:
            image_url = np.nan
        posters.append(image_url)
        
        # Scraping the movie's genre
        genre = container.find("span", class_="genre").text.rstrip().replace("\n","").split(",")
        genres.append(genre)
        
        # Scraping the movie's original_language
        original_language = 'en-US'
        original_languages.append(original_language)
        
        # Scraping the movie's overview
        overview = container.find_all("p", class_="text-muted")[-1].text.lstrip()
        overviews.append(overview)
        
        # Scraping the movie's name
        name = container.h3.a.text
        titles.append(name)
        
        # Scraping the movie's year
        release_date = re.sub(r"[()]","", container.h3.find('span', class_='lister-item-year').text)
        release_dates.append(release_date)
        
        # Scraping the movie's length
        runtime = container.find('span', class_='runtime').text if container.p.find('span', class_='runtime') else '-'
        runtimes.append(runtime)
        
        # Scraping the rating
        imdb = float(container.strong.text)
        imdb_ratings.append(imdb)
        
        # Scraping the metascore
        m_score = container.find('span', class_='metascore').text if container.find('span', class_='metascore') else '-'
        metascores.append(m_score)
        
        # Scraping votes and gross earnings
        nv = container.find_all('span', attrs={'name':'nv'})
        budget = nv[1].text if len(nv) > 1 else '-'
        budgets.append(budget)
        vote_count = nv[0].text.replace(',','')
        vote_counts.append(vote_count)
        
        # scraping casts,movieDirector,movieStars
        movieCast = container.find("p", class_="")
        try:
            casts = movieCast.text.replace("\n","").split('|')
            casts = [x.strip() for x in casts]
            casts = [casts[i].replace(j, "") for i,j in enumerate(["Director:", "Stars:"])]
            movieDirector.append(casts[0])
            movieStars.append([x.strip() for x in casts[1].split(",")])
        except:
            casts = movieCast.text.replace("\n","").strip()
            movieDirector.append(np.nan)
            movieStars.append([x.strip() for x in casts.split(",")])
        
        # calculate popularity 
        popularitys = (float(vote_count) / (float(vote_count) + minVote_count)) * imdb + (minVote_count / (float(vote_count) + minVote_count)) * meanImdb_rating
        popularity.append(popularitys)

In [6]:
movies = pd.DataFrame({
    'id':ids,
    'budget':budgets,
    'genre':genres,
    'original_language':original_languages,
    'overviews':overviews,
    'release_date':release_dates,
    'runtime':runtimes,
    'titles':titles,
    'movieStars':movieStars,
    'movieDirector':movieDirector,
    'url':urls,
    'poster':posters,
    'vote_average':imdb_ratings,
    'metascore':metascores,
    'vote_count':vote_counts,
    'popularity':popularity
})

In [7]:
movies.to_csv('movies1000.csv', index=False)

In [8]:
movies

,id,budget,genre,original_language,overviews,release_date,runtime,titles,movieStars,movieDirector,url,poster,vote_average,metascore,vote_count,popularity
0,tt7131622,$142.50M,"[Comedy, Drama]",en-US,A faded television actor and his stunt double ...,2019,161 min,Once Upon a Time... In Hollywood,"[Leonardo DiCaprio, Brad Pitt, Margot Robbie, ...",Quentin Tarantino,http://www.imdb.com/title/tt7131622,https://m.media-amazon.com/images/M/MV5BOTg4ZT...,7.6,83,608180,7.596564
1,tt4154796,$858.37M,"[Action, Adventure, Drama]",en-US,After the devastating events of Avengers: Infi...,2019,181 min,Avengers: Endgame,"[Robert Downey Jr., Chris Evans, Mark Ruffalo,...","Directors:Anthony Russo, Joe Russo",http://www.imdb.com/title/tt4154796,https://m.media-amazon.com/images/M/MV5BMTc5MD...,8.4,78,908716,8.395064
2,tt12361974,-,"[Action, Adventure, Fantasy]",en-US,Determined to ensure Superman's ultimate sacri...,2021,242 min,Zack Snyder's Justice League,"[Henry Cavill, Ben Affleck, Gal Gadot, Amy Adams]",Zack Snyder,http://www.imdb.com/title/tt12361974,https://m.media-amazon.com/images/M/MV5BYjI3ND...,8.1,54,316599,8.088736
3,tt0111161,$28.34M,[Drama],en-US,Two imprisoned men bond over a number of years...,1994,142 min,The Shawshank Redemption,"[Tim Robbins, Morgan Freeman, Bob Gunton, Will...",Frank Darabont,http://www.imdb.com/title/tt0111161,https://m.media-amazon.com/images/M/MV5BMDFkYT...,9.3,80,2426954,9.297037
4,tt0068646,$134.97M,"[Crime, Drama]",en-US,An organized crime dynasty's aging patriarch t...,1972,175 min,The Godfather,"[Marlon Brando, Al Pacino, James Caan, Diane K...",Francis Ford Coppola,http://www.imdb.com/title/tt0068646,https://m.media-amazon.com/images/M/MV5BM2MyNj...,9.2,100,1679909,9.195900
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,tt0119472,$0.00M,"[Action, Crime, Comedy]",en-US,Two terminally ill patients escape from a hosp...,1997,87 min,Knockin' on Heaven's Door,"[Til Schweiger, Jan Josef Liefers, Thierry van...",Thomas Jahn,http://www.imdb.com/title/tt0119472,https://m.media-amazon.com/images/M/MV5BMTk2Mj...,7.9,-,28477,7.804692
996,tt2213054,$1.12M,"[Drama, Sport]",en-US,Three friends growing up in India at the turn ...,2013,120 min,Kai Po Che,"[Amit Sadh, Sushant Singh Rajput, Rajkummar Ra...",Abhishek Kapoor,http://www.imdb.com/title/tt2213054,https://m.media-amazon.com/images/M/MV5BMTgwNT...,7.8,40,34408,7.727823
997,tt0456144,$2.22M,"[Comedy, Drama, Romance]",en-US,Munna Bhai embarks on a journey with Mahatma G...,2006,144 min,Lage Raho Munna Bhai,"[Sanjay Dutt, Arshad Warsi, Vidya Balan, Boman...",Rajkumar Hirani,http://www.imdb.com/title/tt0456144,https://m.media-amazon.com/images/M/MV5BN2ZmMD...,8.0,-,44457,7.930463
998,tt0270053,-,"[Comedy, Drama]",en-US,Lives of residents in a small Anatolian villag...,2001,110 min,Vizontele,"[Yilmaz Erdogan, Demet Akbag, Altan Erkekli, C...","Directors:Yilmaz Erdogan, Ömer Faruk Sorak",http://www.imdb.com/title/tt0270053,https://m.media-amazon.com/images/M/MV5BMjA2Mz...,8.0,-,34761,7.912608


In [12]:
from IPython.display import Image
from IPython.core.display import HTML 
from IPython.display import display, Image

for i in range(10):
    i=Image(url= movies['poster'][i])
    display(i)